### ECON-T Scripts

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import genfromtxt
import os
import matplotlib.colors as mcolors
from ast import literal_eval
from scipy.optimize import curve_fit
import pandas as pd
import matplotlib.patches as mpatches
import sys

#### PRBS Scan: can do average and std dev

In [ ]:
def data_load(nTrials,vSet,vSetFirstDig,boardNum,runNum):
    variability = np.zeros((15,12))
    average = np.zeros((15,12))
    test = np.zeros((nTrials,15,12))
    for i in range(nTrials):
       temp1 = genfromtxt(f"/Users/alexcampbell/documents/research/Fermilab/ECONT_testing/board{boardNum}_run{runNum}/{vSet}_{i}_prbs_counters_scan_0.05s_1.2V_999board.csv", delimiter=',')
       temp2 = temp1[1:,:] 
       test[i][:][:] = temp2 
    
    for i in range(15):
        for j in range(12):
            var1 = []
            for k in range(nTrials):
                var1.append(test[k][i][j])
            var1 = np.array(var1)
            variability[i][j] = np.std(var1)
            average[i][j] = np.average(var1)
    
    ## plots the Std dev in transmission errors rate
    a,b=np.meshgrid(np.arange(12),np.arange(15))
    norm = mcolors.TwoSlopeNorm(vmin=0, vmax = 255, vcenter=.9)
    plt.hist2d(a.flatten(),b.flatten(),weights=variability.flatten(),bins=(np.arange(13),np.arange(15)),cmap='bwr');
    plt.rcParams.update({'font.size': 32})
    plt.colorbar().set_label(label='Std. Dev. in transmission errors rate ',size=32)
    plt.title(f"1.28 GHz input phase alignment, {vSetFirstDig}V", size=32)
    plt.ylabel('Phase Select Setting', size=32)
    plt.xlabel('Channel Number', size=32)
    plt.savefig(f'/Users/alexcampbell/documents/research/Fermilab/ECONT_testing/board{boardNum}_run{runNum}/plots/{vSet}_variance_in_errors.png',dpi=300, facecolor = "w")
    plt.clf()
    
    ## plots the average of the transmission errors rate
    plt.hist2d(a.flatten(),b.flatten(),weights=average.flatten(),bins=(np.arange(13),np.arange(15)),cmap='bwr',norm = norm);
    plt.rcParams.update({'font.size': 32})
    plt.colorbar().set_label(label='Data (PRBS) transmission errors rate ',size=32)
    plt.title(f"1.28 GHz input phase alignment, {vSetFirstDig}V", size=32)
    plt.ylabel('Phase Select Setting', size=32)
    plt.xlabel('Channel Number', size=32)
    plt.savefig(f'/Users/alexcampbell/documents/research/Fermilab/ECONT_testing/board{boardNum}_run{runNum}/plots/{vSet}_phase_scan_single_color.png',dpi=300, facecolor = "w")
    plt.clf()
    
## Ntrials is just the number of trials
## vSet, vSetFirstDig, boardNum, and Run num are just for naming files and loading files
## This can be changed

#### Delay Scan

In [ ]:
def delayData(nTrials,vSet,vSetFirstDig,boardNum,runNum):
    delayVar = np.zeros((13,128))
    delayAverage = np.zeros((13,128))
    test = np.zeros((nTrials,13,128))
    for i in range(nTrials):
        mainlist=[]
        with open(f"/Users/alexcampbell/documents/research/Fermilab/ECONT_testing/board{boardNum}_run{runNum}/{vSet}_{i}_delay_scan_errors.txt") as f:
            mainlist = [list(literal_eval(line)) for line in f]
        dataDelay = np.array(mainlist)
        test[i][:][:] = dataDelay
    for i in range(test.shape[1]):
        for j in range(test.shape[2]):
            var1 = []
            for k in range(nTrials):
                var1.append(test[k][i][j])
            var1 = np.array(var1)
            delayVar[i][j] = np.std(var1)
            delayAverage[i][j] = np.average(var1)
            
     ## plots the Std dev in transmission errors rate
    a,b=np.meshgrid(np.arange(13),np.arange(128))
    plt.hist2d(a.flatten(),b.flatten(),weights=delayVar.T.flatten(),bins=(np.arange(14),np.arange(129)),cmap='bwr');
    plt.colorbar().set_label(label='Std. Dev. in transmission errors rate ',size=11)
    plt.title(f"1.28 GHz input phase alignment, {vSetFirstDig}V", size=12)
    plt.ylabel('Phase Select Setting', size=13)
    plt.xlabel('Channel Number', size=13)
    plt.savefig(f'/Users/alexcampbell/documents/research/Fermilab/ECONT_testing/board{boardNum}_run{runNum}/plots/{vSet}_delay_variance_in_errors.png',dpi=300, facecolor = "w")
#     plt.clf()
    plt.figure()
    
    ## plots the average of the transmission errors rate
    plt.hist2d(a.flatten(),b.flatten(),weights=delayAverage.T.flatten(),bins=(np.arange(14),np.arange(129)),cmap='bwr');
    plt.colorbar().set_label(label='Data (delay scan) transmission errors rate ',size=11)
    plt.title(f"1.28 GHz input phase alignment, {vSetFirstDig}V", size=12)
    plt.ylabel('Phase Select Setting', size=13)
    plt.xlabel('Channel Number', size=13)
    plt.savefig(f'/Users/alexcampbell/documents/research/Fermilab/ECONT_testing/board{boardNum}_run{runNum}/plots/{vSet}_delay_scan_single_color.png',dpi=300, facecolor = "w")
    #plt.clf()
    
    
### Arguments for the function are the same for the prbs scan and this also computes the average
### and std dev

#### PLL Scan

In [ ]:
def pllECONT(indir, fname, scale):
    mainlist=[]
    with open(f"{indir}/{fname}.txt") as f:
        mainlist = [list(literal_eval(line)) for line in f]
    pllSettings = np.array(mainlist)
    b,a=np.meshgrid(np.arange(35,44,(1/scale)),np.arange(56))
    plt.hist2d(b.flatten(),a.flatten(),weights=pllSettings.T.flatten(),bins=(np.arange(34,45,(1/scale)),np.arange(57)),cmap='Blues')
    plt.xlabel('Reference Clock Frequency Setting (MHz)', size=32)
    plt.ylabel('CapBank Select Setting', size=32)
    handles, labels = plt.gca().get_legend_handles_labels()
    patch = mpatches.Patch(color='#08306b', label='PLL Lock with VCO Override')
    handles.append(patch)
    #ax.xaxis.set_ticks(np.arange(0, 50, (1/32))
    plt.legend(handles=handles, loc='lower left')
    plt.title("ECON-T-P1")
    plt.axvline(40)
    plt.rcParams.update({'font.size': 15})
    plt.xlim([35,50])
    plt.savefig(f'{indir}/PLLSetting_1_{scale}_2_flipped_axes.png',dpi=300, facecolor = "w")
    # plt.plot(d[0][:],e[0][:], label = "1.08 V")
    # plt.plot(d[1][:],e[1][:], label = "1.20 V")
    # plt.plot(d[2][:],e[2][:], label = "1.32 V")
    # plt.legend(title="ASIC 18")
    plt.figure()
## indir is the input directory
## fname is the data file name
## scale just means the denominator in 1/(step size)

### ECON-D Plot Scripts

#### PRBS Scan 

In [ ]:
def prbsECOND(indir, fname):
    ### indir is the input directory
    ### fname is the data file name
    new_data = np.load(f"{indir}/{fname}.npy")
    a,b=np.meshgrid(np.arange(12),np.arange(15))
    norm = mcolors.TwoSlopeNorm(vmin=0, vmax = 255, vcenter=.9)
    plt.hist2d(a.flatten(),b.flatten(),weights=new_data.flatten(),bins=(np.arange(13),np.arange(16)),cmap='bwr',norm=norm);
    plt.colorbar().set_label(label='Data (PRBS) transmission errors rate ',size=32)
    #plt.title(f"1.28 GHz input phase alignment, 1.20V", size=32)
    #plt.title("Phase Scan for ECON-T-P1 at 40MHz",size="28")
    plt.ylabel('Phase Select Setting', size=32)
    plt.xlabel('Channel Number', size=32)
    plt.savefig(f'{indir}/prbs_counters_scan.png',dpi=300, facecolor = "w")

#### PLL Scan

In [ ]:
def econd_pLL_test(boardNum,scale,indir,fname):
    ## boardNum is the board number
    ## scale is the same as before 
    ## load the data
    data2 = genfromtxt(f"{indir}/{fname}_autolocks.csv", delimiter=',')
    data = pd.read_csv(f"{indir}/{fname}_locks.csv", header=None,skiprows=1).to_numpy()
    ## fname is this case is the extension before _autolocks or _locks
    
    ## Set some variables
    allowed_cap_bank_vals=np.array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
                                  13,  14,  15,  24,  25,  26,  27,  28,  29,  30,  31,  56,  57,
                                  58,  59,  60,  61,  62,  63, 120, 121, 122, 123, 124, 125, 126,
                                  127, 248, 249, 250, 251, 252, 253, 254, 255, 504, 505, 506,       507,
                                  508, 509, 510, 511])

    capSettings = np.zeros(len(data2[1]))
    a = [] ## list of cap settings where it locks
    b = [] ## list of frequencies that correspond to a


    for i in range (len(data2[1])):
        for j in range(56):
            if data2[1][i] == allowed_cap_bank_vals[j]:
                capSettings[i] = j

    for i in range(len(data2[1])):
        if data2[2][i] == 1:
            a.append(capSettings[i])
            b.append(data2[0][i])

    x = np.array(a)
    y = np.array(b)
    
    
    ## Make the plots
    b,a=np.meshgrid(np.arange(40,50,(1/scale)),np.arange(56))
    plt.hist2d(b.flatten(),a.flatten(),weights=data.T.flatten(),bins=(np.arange(40,51,(1/scale))-(0.5/scale),np.arange(57)-0.5),cmap='Blues', label ="PLL Lock")
    plt.scatter((y/8),x,color="red",label="Automatic Lock")
    #plt.colorbar().set_label(label='PLL Locking Status',size=32)
    handles, labels = plt.gca().get_legend_handles_labels()
    patch = mpatches.Patch(color='#08306b', label='PLL Lock with VCO Override')
    handles.append(patch) 

    plt.legend(handles=handles, loc='lower left')
    plt.xlabel('Frequency Setting (MHz)', size=32)
    plt.ylabel('CapBank Select Setting', size=32)
    #plt.title(f'Board Number {boardNum}')
    plt.title("ECON-D-P1 Board 5 PLL Scan")
    plt.xlim([35,50])
    plt.savefig(f'{indir}/ECOND-PLL-1_{scale}_BoardNum{boardNum}.png',dpi=300, facecolor = "w")
    print()

    